In [ ]:
!pip install  accelerate transformers datasets evaluate seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 37.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("jnlpba")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18546 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3856 [00:00<?, ? examples/s]

In [ ]:
raw_datasets["train"][0]

{'id': '1',
 'tokens': ['IL-2',
  'gene',
  'expression',
  'and',
  'NF-kappa',
  'B',
  'activation',
  'through',
  'CD28',
  'requires',
  'reactive',
  'oxygen',
  'production',
  'by',
  '5-lipoxygenase',
  '.'],
 'ner_tags': [1, 2, 0, 0, 9, 10, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0]}

In [ ]:
listing_label = raw_datasets["train"].features[f"ner_tags"].feature.names
listing_label

['O',
 'B-DNA',
 'I-DNA',
 'B-RNA',
 'I-RNA',
 'B-cell_line',
 'I-cell_line',
 'B-cell_type',
 'I-cell_type',
 'B-protein',
 'I-protein']

## preprocessing

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
data = raw_datasets["train"][0]
tokenized_input = tokenizer(data["tokens"],is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'il',
 '-',
 '2',
 'gene',
 'expression',
 'and',
 'n',
 '##f',
 '-',
 'kappa',
 'b',
 'activation',
 'through',
 'cd',
 '##28',
 'requires',
 'reactive',
 'oxygen',
 'production',
 'by',
 '5',
 '-',
 'lip',
 '##ox',
 '##y',
 '##genase',
 '.',
 '[SEP]']

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_data = raw_datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/18546 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/3856 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
import numpy as np

labels = [listing_label[i] for i in data[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [listing_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [listing_label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2label = { i:k for i, k in enumerate(listing_label) }
label2id = { v:k for k, v in id2label.items() }

In [ ]:
id2label

{0: 'O',
 1: 'B-DNA',
 2: 'I-DNA',
 3: 'B-RNA',
 4: 'I-RNA',
 5: 'B-cell_line',
 6: 'I-cell_line',
 7: 'B-cell_type',
 8: 'I-cell_type',
 9: 'B-protein',
 10: 'I-protein'}

In [ ]:
label2id

{'O': 0,
 'B-DNA': 1,
 'I-DNA': 2,
 'B-RNA': 3,
 'I-RNA': 4,
 'B-cell_line': 5,
 'I-cell_line': 6,
 'B-cell_type': 7,
 'I-cell_type': 8,
 'B-protein': 9,
 'I-protein': 10}

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:


from transformers import TrainingArguments, Trainer

# Setting training arguments
training_args = TrainingArguments(
    output_dir="ner_on_proteindata",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    report_to="tensorboard",
)

In [ ]:
columns = ["attention_mask", "input_ids", "labels", "token_type_ids"]
batch_size = 16

In [ ]:
# Creating a trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],  # Use get() to avoid KeyError
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:

# Training the model
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.195500,0.201951,0.638845,0.753752,0.691558,0.931462
2,0.154100,0.192692,0.655234,0.763103,0.705067,0.936302


TrainOutput(global_step=2320, training_loss=0.21004213793524382, metrics={'train_runtime': 319.7779, 'train_samples_per_second': 115.993, 'train_steps_per_second': 7.255, 'total_flos': 759750435341772.0, 'train_loss': 0.21004213793524382, 'epoch': 2.0})

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1713717942.4f5238951b92.788.0:   0%|          | 0.00/7.08k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Galaxyman/ner_on_proteindata/commit/e9fbe19949eda7deabf65125ce69533a3ed61900', commit_message='End of training', commit_description='', oid='e9fbe19949eda7deabf65125ce69533a3ed61900', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
text = "Structural proteins like actin and tubulin provide the scaffolding for cells while enzymatic proteins like kinases and phosphatases"
from transformers import pipeline

classifier = pipeline("feature-extraction", model="Galaxyman/ner_on_proteindata")
classifier(text)

[[[-0.28371676802635193,
   -0.943052351474762,
   0.6754981279373169,
   0.19684149324893951,
   0.16017910838127136,
   -0.27720847725868225,
   -0.37415811419487,
   1.441422462463379,
   -0.2937760055065155,
   0.24341556429862976,
   -0.0994846373796463,
   -0.20602136850357056,
   0.7562535405158997,
   0.2784421145915985,
   0.3254067599773407,
   -0.5879138708114624,
   -0.637360692024231,
   -0.488822340965271,
   -0.7007059454917908,
   0.9933807849884033,
   -0.20241422951221466,
   0.03138703480362892,
   0.8754459619522095,
   -0.4148978590965271,
   0.4917653203010559,
   -1.9346849918365479,
   -0.6489612460136414,
   0.7573553919792175,
   0.4901174008846283,
   0.3499705195426941,
   -1.1508105993270874,
   1.0850259065628052,
   0.574442446231842,
   0.4756728708744049,
   0.3953995108604431,
   -0.4592421054840088,
   -0.35277512669563293,
   0.17629407346248627,
   -0.5700019598007202,
   1.0887624025344849,
   -0.15356852114200592,
   -0.5580165982246399,
   0.0596